In [26]:
import pandas as pd
from scipy.stats import ks_2samp


In [21]:
training_file_path= "artifact\\07_28_2023_23_55_38\\data_ingestion\\ingested\\train.csv"

In [22]:
testing_file_path= 'artifact\\07_28_2023_23_55_38\\data_ingestion\\ingested\\test.csv'

In [23]:
train_df= pd.read_csv(training_file_path)
test_df= pd.read_csv(testing_file_path)

In [24]:
train_df.columns

Index(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
       'ag_000', 'ag_001', 'ag_002',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=171)

In [25]:
test_df.columns

Index(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
       'ag_000', 'ag_001', 'ag_002',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=171)

In [37]:
def get_data_drift_report(base_df, current_df, threshold= 0.5):
    report= {}
    for column in base_df.columns:
        b_df= base_df[column]
        c_df= current_df[column]
        is_same_dist= ks_2samp(b_df, c_df)
        if threshold<=is_same_dist.pvalue:
            is_found= False
        else:   
            is_found= True
        report.update({
        column: {
        'p_value': float(is_same_dist.pvalue),
        'drift_status': is_found}
            })
    return report

In [38]:
drift_report= get_data_drift_report(train_df, test_df)
drift_report

{'class': {'p_value': 1.0, 'drift_status': False},
 'aa_000': {'p_value': 0.2839471070260873, 'drift_status': True},
 'ab_000': {'p_value': 0.8390566237463858, 'drift_status': False},
 'ac_000': {'p_value': 0.25293908047657, 'drift_status': True},
 'ad_000': {'p_value': 0.3083152741639448, 'drift_status': True},
 'ae_000': {'p_value': 1.0, 'drift_status': False},
 'af_000': {'p_value': 1.0, 'drift_status': False},
 'ag_000': {'p_value': 0.9994970486422927, 'drift_status': False},
 'ag_001': {'p_value': 0.9999437548422836, 'drift_status': False},
 'ag_002': {'p_value': 0.9999967649890598, 'drift_status': False},
 'ag_003': {'p_value': 0.9997015835984276, 'drift_status': False},
 'ag_004': {'p_value': 0.7507699628161292, 'drift_status': False},
 'ag_005': {'p_value': 0.3463281731849206, 'drift_status': True},
 'ag_006': {'p_value': 0.6035893912644353, 'drift_status': False},
 'ag_007': {'p_value': 0.2746206480550112, 'drift_status': True},
 'ag_008': {'p_value': 0.3836372698378495, 'drif

In [39]:
from sensor.utils.main_utils import write_yaml_file
import os, sys

In [40]:
write_yaml_file(file_path= os.path.join(os.getcwd(), 'report.yaml'), content= drift_report, replace= True)